Deriving Probability of Shaking Exceedance from Source Earthquakes
====


The goal here is to take an array of shaking source magnitudes and convert them to an array of rates of exceeding some shaking threshold $Y_t$.  Combined with a model of underlying processes (eg. Poissonian) we can turn an average rate into a probability of exceedance over some time period.  The main steps are:

1. Using a set of ground motion prediction equations (We are using Cua \& Heaton), take magnitudes and distances to predict the average shaking contribution from each source to each spatial bin
* Of course, due to uncertainties, this is merely the *average* shaking associated with these distance/magnitude inputs.  The uncertainty in these shaking equations is typically log-normal. To find the probability that this source magnitude, at this distance, will cause shaking exceeding our threshold $Y_t$, we integrate the uncertainty distribution from $Y_t \rightarrow \infty$ for each spatial bin
* We can now multiply this array of probabilities by the rate that such a source earthquake occurs.  This yields the rate of exceedance for this one source magnitude.Repeat the process for each source magnitude, summing together the exceedance rate contributions from each source
* If desired, convert each bins total rate of exceedance to a probability using an assumed probability distribution for the events, e.g. Poissonian

In [5]:
import numpy as np
import math
from scipy.integrate import quad

motion_type_prams_lst_vars = ['a', 'b', 'c1', 'c2', 'd', 'e', 'sig']
motion_type_prams_lst = {'PGA-rock':[0.73, -7.2e-4, 1.16, 0.96, -1.48, -0.42, 0.31],
					 'PGA-soil':[0.71, -2.38e-3, 1.72, 0.96, -1.44, -2.45e-2, 0.33],
					 'PGV-rock':[0.86, -5.58e-4, 0.84, 0.98, -1.37, -2.58, 0.28],
					 'PGV-soil':[0.89, -8.4e-4, 1.39, 0.95, -1.47, -2.24, 0.32]}
motion_type_prams = {key:{ky:vl for ky,vl in zip(motion_type_prams_lst_vars, vals)} for key,vals in motion_type_prams_lst.items()}

1. GMPEs
----------

Here we reference Cua & Heaton (2009) ground motion equations.  For now, we simplify our task and only focus on peak ground acceleration (PGA) for horizontal motion associated with S-waves on soil.  

We begin by considering some source earthquake with magnitude magnitude $M$ and rate of occurance $r$.  We wish to calculate its shaking contribution to some spatial cell a distance $R$ away from the epicenter.

For our acceleration $Y$:

\begin{equation}
\log(Y) = aM+b(L_1+C(M)) + d\log\left(L_1 + C(M) + e \right)
\end{equation}

Where
$$L_1 = \sqrt{L^2+9}$$
and 
$$C(M) = c_1\exp\left( (c_2(M-5) \times \left(\arctan(M-5)+\frac{\pi}{2}\right)\right)$$

Fitting parameters $a$, $b$, $c_1$, $c_2$, $d$, and $e$ are given below.

| $$a$$ | $$b$$ | $$c_1$$ | $$c_2$$ | $$d$$ | $$e$$ | $$\sigma$$ |
|-----|---------|---------|---------|-------|-------|------------|
| 0.71 | -2.38e-3 | 1.72 | 0.96 | -1.44 | -2.45e-2 | 0.33 |


In [ ]:
def f_Y(L,M, a=None, b=None, c1=None, c2=None, d=None, e=None, sig=None, motion_type='PGA-soil'):
    if motion_type!=None:
    return f_Y(R,M, motion_type=None, **motion_type_prams[motion_type])
    else:
        return 10**(a*M + b*(np.sqrt(L**2+9) + C(M, c1, c2)) + d*np.log10(np.sqrt(L**2+9) + C(M, c1, c2)) + e)

def C(M, c1, c2):
    return c1*np.exp(c2*(M-5))*(np.arctan(M-5)+np.pi/2.0)

2. Probability of Exceeding Threshold for Single Source
----------

Our calculated $Y$ is the average shaking associated with $M$ and $L$.  The unvertainty follows a log-normal distribution:

$$P(\log(Y')) = \frac{1}{\sqrt{2\pi}\sigma}\exp\left( \frac{-(\log(Y')-\log(Y))^2}{2\sigma} \right)$$ 

Where $\sigma$ is the standard deviation of the distribution, also given in the table above.

To compute the total probability of shaking exceeding $Y_t$, we integrate:

$$P(>\log(Y_t)) = \int_{Y_t}^{\infty}P(\log(Y'))dY' $$

In [ ]:
def int_log_norm(Y, threshold, motion_type):
    result = quad(normal_integrand, np.log10(threshold), np.inf, args=(np.log10(Y), motion_type_prams[motion_type]['sig']))
    return result[0]

def normal_integrand(x, mean, sig):
    return 1/(np.sqrt(2*np.pi)*sig)*np.exp(-((x-mean)/sig)**2/2)

3. Rate of Exceedance
---

The above calculated exceedance probability will contribute every time such an earthquake $i$ occurs, which means this single (repeating) source's overall contribution to the *rate* of shaking exceedance is simply 

$$R_i = r_i \times P_i(>\log(Y_t)) $$


The total rate of shaking exceedance, given a set of N source events, is

$$R_{tot} = \sum_{i}^{N} R_i$$


4. Total Probability of Exceedance for a Given Time Period
---

Assuming Poissonian statistics, this rate of exceedance $R_{tot}$ can be converted to a probability of exceedance for some time period $T$ through the Poisson equation:

$$ P(>\log(Y_t), T) = 1-\exp(-R_{tot}T)$$

Source
---

Ned Field wrote an excellent Probabilistic Seismic Hazard Analysis primer here: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.472.4509&rep=rep1&type=pdf

He includes some other sources for a general approach like this.